In [1]:
import zipfile
from lxml import etree
import shutil
import os

In [26]:
INPUT_DOCX = "ПП.docx"
OUTPUT_DOCX = "ПП_filled.docx"
TEMP_DIR = "tmpdoc"


def get_user_input():

    data = {}

    data["student_name"] = input("  ФИО студента:      ").strip()
    data["course"] = input("  Курс:              ").strip()
    data["group"] = input("  Группа:            ").strip()
    data["date_from"] = input("  Дата начала:       ").strip()
    data["date_to"] = input("  Дата окончания:    ").strip()
    data["company"] = input("  Наименование предприятия:  ").strip()
    data["supervisor_university"] = input("  Руководитель от ВШЭ (ФИО): ").strip()
    data["supervisor_company"] = input("  Руководитель от предприятия (ФИО): ").strip()
    data["year"] = input("  Год (для 'Пермь, 20__'):   ").strip()

    return data


def is_empty_underlined_run(run, ns):
    """Проверяет, является ли run пустым подчёркнутым полем."""
    underline = run.find('.//w:u', namespaces=ns)
    text_elem = run.find('.//w:t', namespaces=ns)

    has_underline = underline is not None
    is_empty = text_elem is None or not (text_elem.text and text_elem.text.strip())

    return has_underline and is_empty


def replace_next_empty_field(runs, value, ns):
    for run in runs:
        if is_empty_underlined_run(run, ns):
            # Очищаем содержимое run
            for child in list(run):
                run.remove(child)

            # Вставляем новый текст
            text_elem = etree.Element("{%s}t" % ns["w"])
            text_elem.text = value
            # Сохраняем пробелы!
            text_elem.set("{http://www.w3.org/XML/1998/namespace}space", "preserve")
            run.append(text_elem)
            return True

    return False

def split_date(date_str):
    """Разбивает дату 'ДД.ММ.ГГГГ' на части [день, месяц, год_без_20]."""
    parts = date_str.split(".")
    if len(parts) == 3:
        day, month, year = parts
        # Год: "2025" → "25" (убираем "20" в начале)
        year_short = year[2:] if len(year) == 4 else year
        return [day, month, year_short]
    return [date_str]

def process_document(data):
    """Обрабатывает документ и заполняет поля."""

    if os.path.exists(TEMP_DIR):
        shutil.rmtree(TEMP_DIR)
    os.mkdir(TEMP_DIR)

    try:
        with zipfile.ZipFile(INPUT_DOCX, 'r') as archive:
            archive.extractall(TEMP_DIR)

        doc_xml_path = os.path.join(TEMP_DIR, "word", "document.xml")
        tree = etree.parse(doc_xml_path)
        root = tree.getroot()
        ns = root.nsmap

        runs = tree.findall('.//w:r', namespaces=ns)

        date_from_parts = split_date(data["date_from"])
        date_to_parts = split_date(data["date_to"])

        year_short = data["year"][2:] if len(data["year"]) == 4 else data["year"]

        fields = [
            # Страница 1 — шапка
            (f" {data['course']} ", "Курс"),                    # #0
            (f" {data['group']} ", "Группа"),                   # #1
            (f" {data['student_name']} ", "ФИО"),               # #2
            ("", "пусто"),                                      # #3
            ("", "пусто"),                                      # #4
            (f" {date_from_parts[0]}", "день начала"),          # #5
            (date_from_parts[1], "месяц начала"),               # #6
            (f"{date_from_parts[2]} ", "год начала"),           # #7
            (f"{date_to_parts[2]} ", "год конца"),              # #8

            # Даты подписей (#9-13)
            ("", "пусто"),                                      # #9
            ("", "пусто"),                                      # #10
            ("", "пусто"),                                      # #11
            ("", "пусто"),                                      # #12
            ("", "пусто"),                                      # #13

            # Выполнил студент группы
            (f" {data['group']} ", "группа студента"),          # #14

            # Пермь, 20__
            (year_short, "год Пермь 1"),                        # #15
            (year_short, "год Пермь 2"),                        # #16

            # Страница 2 — курс и группа
            (f" {data['course']} ", "Курс 2"),                  # #17
            (f" {data['group']} ", "Группа 2"),                 # #18

            # Срок практики (страница 2)
            (f" {date_from_parts[0]}", "день начала 2"),        # #19
            (date_from_parts[1], "месяц начала 2"),             # #20
            (f"{date_from_parts[2]} ", "год начала 2"),         # #21
            (f"{date_to_parts[2]} ", "год конца 2"),            # #22

            # Наименование предприятия
            (f" {data['company']} ", "предприятие"),            # #23

            # Подписи студента (#24-25)
            (f" {data['student_name']} ", "студент подпись"),   # #24
            ("", "пусто"),                                      # #25

            # Руководитель от ВШЭ (#26-28)
            (f" {data['supervisor_university']} ", "рук ВШЭ"),  # #26
            ("", "пусто"),                                      # #27
            ("", "пусто"),                                      # #28

            # Руководитель от предприятия (#29-31)
            (f" {data['supervisor_company']} ", "рук предпр"),  # #29
            ("", "пусто"),                                      # #30
            ("", "пусто"),                                      # #31
        ]
        for value, name in fields:
            if not replace_next_empty_field(runs, value, ns):
                print(f"Предупреждение: поле '{name}' не найдено")

        tree.write(doc_xml_path, encoding="utf-8", xml_declaration=True)

        with zipfile.ZipFile(OUTPUT_DOCX, 'w', zipfile.ZIP_DEFLATED) as archive:
            for folder, _, files in os.walk(TEMP_DIR):
                for file in files:
                    file_path = os.path.join(folder, file)
                    arc_name = os.path.relpath(file_path, TEMP_DIR)
                    archive.write(file_path, arc_name)

        return True

    finally:
        if os.path.exists(TEMP_DIR):
            shutil.rmtree(TEMP_DIR)

In [27]:

if __name__ == "__main__":

    # Проверка существования исходного файла
    if not os.path.exists(INPUT_DOCX):
        print(f"Ошибка: файл '{INPUT_DOCX}' не найден")
        exit(1)

    # Получение данных от пользователя
    data = get_user_input()
    if data is None:
        exit(0)

    # Обработка документа
    print("  Обработка...")

    if process_document(data):
        print(f"Файл сохранён: {OUTPUT_DOCX}")
    else:
        print("Ошибка при обработке документа")
        exit(1)

  ФИО студента:      Скачко Владислава Денисовна
  Курс:              4
  Группа:            РИС-22-4
  Дата начала:       01.01.2025
  Дата окончания:    01.02.2025
  Наименование предприятия:  НИУ ВШЭ
  Руководитель от ВШЭ (ФИО): Руководитель
  Руководитель от предприятия (ФИО): Руководитель
  Год (для 'Пермь, 20__'):   25
  Обработка...
Файл сохранён: ПП_filled.docx
